# Lossless -> gridded 

Grids `last_trade` data: 

- Each `BinanceLastTradesGrid` reads the underlying lossless dataset `symbol_date_df` to determine symbol-date pairs. 
- Each dataset maintains a list of "validated" dates. Second runs will be very quick

In [6]:
import mnemosyne as ms 
import polars as pl
from datetime import date as Date
from timedelta_isoformat import timedelta as Timedelta

args = [
    (ms.DatasetType.BinanceSpotTrades, Timedelta(seconds=5), 'USDC'),
    (ms.DatasetType.BinanceUmPerpTrades, Timedelta(seconds=5), 'USDC'),
    (ms.DatasetType.BinanceSpotTrades, Timedelta(minutes=10), 'USDC'),
    (ms.DatasetType.BinanceUmPerpTrades, Timedelta(minutes=10), 'USDC'),
    
    (ms.DatasetType.BinanceSpotTrades, Timedelta(seconds=5), 'USDT'),
    (ms.DatasetType.BinanceUmPerpTrades, Timedelta(seconds=5), 'USDT'),
    (ms.DatasetType.BinanceSpotTrades, Timedelta(minutes=10), 'USDT'),
    (ms.DatasetType.BinanceUmPerpTrades, Timedelta(minutes=10), 'USDT'),
]

In [8]:
for dataset_type, grid_interval, peg_symbol in args:
    # Reads the underlying lossless dataset's `symbol_date_df` to determine symbol-date pairs
    dataset = ms.binance.BinanceLastTradesGrid(
        peg_symbol=peg_symbol, 
        grid_interval=grid_interval, 
        dataset_type=dataset_type, 
        parquet_names='*.parquet',
        num_workers=1, 
    )
    print(f'{dataset_type} {grid_interval} {peg_symbol}: {dataset.path}')
    dataset.compute(recompute=False, days_per_batch=60)

BinanceSpotTrades 0:00:05 USDC: /data/mnemosyne/binance/grids/spot/last_trade/PT5S/peg_symbol=USDC
BinanceUmPerpTrades 0:00:05 USDC: /data/mnemosyne/binance/grids/futures/um/last_trade/PT5S/peg_symbol=USDC
BinanceSpotTrades 0:10:00 USDC: /data/mnemosyne/binance/grids/spot/last_trade/PT10M/peg_symbol=USDC
BinanceUmPerpTrades 0:10:00 USDC: /data/mnemosyne/binance/grids/futures/um/last_trade/PT10M/peg_symbol=USDC
BinanceSpotTrades 0:00:05 USDT: /data/mnemosyne/binance/grids/spot/last_trade/PT5S/peg_symbol=USDT
BinanceUmPerpTrades 0:00:05 USDT: /data/mnemosyne/binance/grids/futures/um/last_trade/PT5S/peg_symbol=USDT
BinanceSpotTrades 0:10:00 USDT: /data/mnemosyne/binance/grids/spot/last_trade/PT10M/peg_symbol=USDT
BinanceUmPerpTrades 0:10:00 USDT: /data/mnemosyne/binance/grids/futures/um/last_trade/PT10M/peg_symbol=USDT


# Usage example

In [ ]:
peg_symbol = 'USDT'
dstype = ms.DatasetType.BinanceUmPerpTrades
grid_interval = Timedelta(minutes=2)

dataset = ms.binance.BinanceLastTradesGrid(
        peg_symbol=peg_symbol, 
        grid_interval=grid_interval, 
        dataset_type=dstype, 
        parquet_names='*.parquet', 
        num_workers=4, 
    )

# Reading the full dataset
lf = pl.scan_parquet(dataset.path / f'**/{dataset.parquet_names}')
dataset.universe_df